# 17. 多进程

# 多线程替代方案——多进程

## subprocess - 子进程管理
- subprocess 模块允许你生成新的进程，连接它们的输入、输出、错误管道，并且获取它们的返回码。
- 完全跳过线程，使用进程。
- 是派生进程的主要替代方案。

## multiprocessing - 基于进程的并行
- multiprocessing 是一个用于产生进程的包，允许为多核或者多CPU派生进程，具有与 threading 模块相似API（应用程序接口）。
- 使用threading接口派生，使用子进程。
- multiprocessing 包同时提供本地和远程并发，使用子进程代替线程，有效避免 Global Interpreter Lock 带来的影响。因此， multiprocessing 模块允许程序员充分利用机器上的多核。

## concurrent.futures - 启动并行任务
- concurrent.futures 模块是新的异步执行模块，提供异步执行回调高层接口。
- 是任务级别的操作

# 多进程
- 进程间通信（Interprocess Communication）
- 进程之间**无任何共享资源**
- 进程的创建：
    - 直接生成Process实例对象，示例1
    - 派生子类，示例2
- 在OS中查看pid（Process ID，进程ID）、ppid（Parent Process ID，父进程ID）以及它们的关系，示例3
- 生产者消费者模型，示例4
    - JoinableQueue：JoinableQueue 类是 Queue 的子类，额外添加了 task_done() 和 join() 方法。
    - 队列中哨兵的使用，示例4-1
    - 哨兵的改进，示例4-2
    - 哨兵的值可以随意设置，只用来做判断而已

## 进程锁（LOCK）
- 使用 multiprocessing 包中的 Process 模块内置的进程锁
- 用法和多线程类似，但使用函数不同

In [6]:
# 进程创建 示例1
from time import sleep, ctime
# 导入多进程模块
import multiprocessing


def clock(interval):
    while True:
        print("The time is %s" % ctime())
        sleep(interval)

        
if __name__ == '__main__':
    p = multiprocessing.Process(target = clock, args = (5,))
    p.start()
    p.join()

In [8]:
# 进程创建 示例2
import multiprocessing
from time import sleep, ctime


class ClockProcess(multiprocessing.Process):
    '''
    两个函数比较重要
    1. init构造函数
    2. run
    '''

    def __init__(self, interval):
        super().__init__()
        self.interval = interval

    def run(self):
        while True:
            print("The time is %s" % ctime())
            sleep(self.interval)


if __name__ == '__main__':
    p = ClockProcess(3)
    p.start()
    p.join()

In [5]:
# 查看进程ID 示例3
from multiprocessing import Process
import os


def info(title):
    print(title)
    print('module name:', __name__)
    # 得到父进程的ID
    print('parent process:', os.getppid())
    # 得到本身进程的ID
    print('process id:', os.getpid())

def func(name):
    info('function func')
    print('hello', name)


if __name__ == '__main__':
    info('main line')
    p = Process(target=func, args=('bob',))
    p.start()
    p.join()

main line
module name: __main__
parent process: 39588
process id: 34296


In [ ]:
# 生产者消费者模型 示例4
import multiprocessing
from time import ctime


def consumer(input_q):
    print("Into consumer:", ctime())
    while True:
        # 处理项
        item = input_q.get()
        print ("pull", item, "out of q") # 此处替换为有用的工作
        input_q.task_done() # 发出信号通知任务完成
    print ("Out of consumer:", ctime()) # 此句未执行，因为q.join()收集到四个task_done()信号后，主进程启动，未等到print此句完成，程序就结束了

def producer(sequence, output_q):
    print ("Into procuder:", ctime())
    for item in sequence:
        output_q.put(item)
        print ("put", item, "into q")
    print ("Out of procuder:", ctime())


# 建立进程
if __name__ == '__main__':
    q = multiprocessing.JoinableQueue()
    # 运行消费者进程
    cons_p = multiprocessing.Process (target = consumer, args = (q,))
    cons_p.daemon = True  # 将 cons_p 设置为一个守护进程，这个操作必须在 start() 之前
    cons_p.start()

    # 生产多个项，sequence代表要发送给消费者的项序列
    # 在实践中，这可能是生成器的输出或通过一些其他方式生产出来
    sequence = [1,2,3,4]
    producer(sequence, q)
    # 等待所有项被处理
    q.join()

Into procuder: Tue Mar 10 06:27:11 2020
put 1 into q
put 2 into q
put 3 into q
put 4 into q
Out of procuder: Tue Mar 10 06:27:11 2020


In [1]:
# 哨兵的使用（当只有一个线程） 示例4-1
import multiprocessing
from time import ctime


# 设置哨兵问题
def consumer(input_q):
    print("Into consumer:", ctime())
    while True:
        item = input_q.get()
        if item is None:
            break
        print("pull", item, "out of q")
    print ("Out of consumer:", ctime()) # 此句执行完成，再转入主进程

def producer(sequence, output_q):
    print ("Into procuder:", ctime())
    for item in sequence:
        output_q.put(item)
        print ("put", item, "into q")
    print ("Out of procuder:", ctime())

    
if __name__ == '__main__':
    q = multiprocessing.Queue()
    cons_p = multiprocessing.Process(target = consumer, args = (q,))
    cons_p.start()

    sequence = [1,2,3,4]
    producer(sequence, q)

    q.put(None) # 哨兵
    cons_p.join()

Into procuder: Tue Mar 10 06:44:36 2020
put 1 into q
put 2 into q
put 3 into q
put 4 into q
Out of procuder: Tue Mar 10 06:44:36 2020


In [2]:
# 哨兵的改进（当有多个线程） 示例4-2
import multiprocessing
from time import ctime

def consumer(input_q):
    print ("Into consumer:", ctime())
    while True:
        item = input_q.get()
        if item is None:
            break
        print("pull", item, "out of q")
    print ("Out of consumer:", ctime())

def producer(sequence, output_q):
    for item in sequence:
        print ("Into procuder:", ctime())
        output_q.put(item)
        print ("Out of procuder:", ctime())

if __name__ == '__main__':
    q = multiprocessing.Queue()
    cons_p1 = multiprocessing.Process (target = consumer, args = (q,))
    cons_p1.start()

    cons_p2 = multiprocessing.Process (target = consumer, args = (q,))
    cons_p2.start()

    sequence = [1,2,3,4]
    producer(sequence, q)

    q.put(None) # 哨兵1
    q.put(None) # 哨兵2

    cons_p1.join()
    cons_p2.join()

Into procuder: Tue Mar 10 06:44:42 2020
Out of procuder: Tue Mar 10 06:44:42 2020
Into procuder: Tue Mar 10 06:44:42 2020
Out of procuder: Tue Mar 10 06:44:42 2020
Into procuder: Tue Mar 10 06:44:42 2020
Out of procuder: Tue Mar 10 06:44:42 2020
Into procuder: Tue Mar 10 06:44:42 2020
Out of procuder: Tue Mar 10 06:44:42 2020


## 进程信号量

## 进程事件

## 进程池

## 子进程与主进程之间的关系

- 正常情况下，主进程会等待子进程结束再结束。因为主进程要负责回收子进程占用的资源。
- 主进程代码执行完毕不代表主进程的结束。
- 设置守护进程之后，守护进程会随着主进程的代码执行完毕而结束，而此时主进程可以还没有结束。